In [ ]:
import pandas as pd
import numpy as np
import os
import sys
from pathlib import Path
import pickle

import matplotlib.pyplot as plt
from cycler import cycler

# Auxiliary functions used to generate plots

In [ ]:
def plot_comparison_metric(competitors: dict[str, pd.DataFrame],
                           reference_competitor : str,
                           column : str,
                           title_plot : str,
                           y_label : str,
                           x_limit : tuple[float, float],
                           save_path : str) :
    ''' 
    This function generates a plot that compares various scoring plugin under some metric as the % of requested
    GPU cluster resources increases.
    '''

    # Set the font size for all plot elements
    plt.rcParams.update({'font.size': 14})
    
    # Plotting
    fig, ax1 = plt.subplots(figsize=(8, 5))
    line_styles = ['-', '--', ':', '-.'] * 2
    colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'orange']
    ax1.set_prop_cycle(cycler('linestyle', line_styles) + cycler('color', colors))
    
    for k, v in competitors.items() :
        if k == reference_competitor: continue
        ax1.plot(v.index, v[column], label=k)
    ax1.plot(competitors[reference_competitor].index, competitors[reference_competitor][column], label=reference_competitor)

    ax1.set_xlim(x_limit)
    ax1.set_xlabel('% datacenter GPU capacity requested by arrived tasks')
    ax1.set_ylabel(y_label)
    ax1.legend(fontsize='small', ncol=2)
    # plt.title(title_plot)
    plt.grid(True)
    plt.tight_layout()

    plt.savefig(save_path, format='pdf')
    
    plt.show()

In [ ]:
def plot_failed_relative(competitors_sched: dict[str, pd.DataFrame], 
                         reference_competitor : str, column_sched : str, title_plot : str) :
    
    fig, ax1 = plt.subplots(figsize=(10, 6))
    line_styles = ['-', '--', ':', '-.'] * 2
    colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'orange']
    ax1.set_prop_cycle(cycler('linestyle', line_styles) + cycler('color', colors))
    
    reference = competitors_sched[reference_competitor]
    for k, v in competitors_sched.items() :
        if k == reference_competitor: continue
        ax1.plot(v.index, v[column_sched] - reference[column_sched], label=k)
    
    ax1.set_xlabel('% datacenter GPU capacity requested by arrived tasks')
    ax1.set_ylabel(f'Difference w.r.t. {reference_competitor}')
    ax1.legend(fontsize='small', ncol=2)
    plt.title(title_plot)
    plt.grid(True)
    plt.tight_layout()
    
    plt.show()

In [ ]:
def plot_energy_savings(competitors_pwr: dict[str, pd.DataFrame], 
                        reference_competitor : str, 
                        column_power : str, 
                        title_plot : str,
                        save_path : str) :

    # Set the font size for all plot elements
    plt.rcParams.update({'font.size': 14})
    
    fig, ax1 = plt.subplots(figsize=(8, 5))
    line_styles = ['-', '--', ':', '-.'] * 2
    colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'orange']
    ax1.set_prop_cycle(cycler('linestyle', line_styles) + cycler('color', colors))
    
    reference = competitors_pwr[reference_competitor]
    for k, v in competitors_pwr.items() :
        if k == reference_competitor: continue
        ax1.plot(v.index, (reference[column_power] - v[column_power]) / reference[column_power] * 100, label=k)

    ax1.set_xlim((0, 1))
    ax1.set_xlabel('% datacenter GPU capacity requested by arrived tasks')
    ax1.set_ylabel(f'Percentage power savings w.r.t. {reference_competitor}')
    ax1.legend(fontsize='small', ncol=2)
    # plt.title(title_plot)
    plt.grid(True)
    plt.tight_layout()

    plt.savefig(save_path, format='pdf')
    
    plt.show()

### Read the parsed results

In [ ]:
dict_pwr_final_res = None
dict_frag_final_res = None
dict_sched_final_res = None
dict_efficiency = None

with open('dict_pwr_final_res.pkl', 'rb') as f:
    dict_pwr_final_res = pickle.load(f)

with open('dict_frag_final_res.pkl', 'rb') as f:
    dict_frag_final_res = pickle.load(f)

with open('dict_sched_final_res.pkl', 'rb') as f:
    dict_sched_final_res = pickle.load(f)

with open('dict_efficiency.pkl', 'rb') as f:
    dict_efficiency = pickle.load(f)

# Generation of plots

In [ ]:
# Generate the directory of the plots, if needed.
dir_plots = './energy_aware_plots/'
if not os.path.exists(dir_plots): os.makedirs(dir_plots)

In [ ]:
reference_competitor = 'FGD'
for level in dict_pwr_final_res.keys() :

    print(f"Generating plots for {level}...")
    
    # Plot the energy savings achieved with some competitor w.r.t. the reference competitor.
    # X-axis represents the arrived workloads in % of GPU resources available in the cluster. 
    plot_energy_savings(dict_pwr_final_res[level], reference_competitor, 
                        "power_cluster", 
                        f"CPU+GPU power savings vs {reference_competitor} (set experiments: {level})",
                        dir_plots + "pwrsaving_" + level + '.pdf')

    plot_comparison_metric(dict_efficiency[level],
                           reference_competitor,
                           'usage_efficiency',
                           f"GPU Allocated vs Requested resource ratio (set experiments: {level})",
                           'GPU Allocated vs Requested resource ratio',
                           (0.8,1),
                           dir_plots + "gpuocc_" + level + '.pdf')



    #### Ignored (for now) ####s
    if False :
        plot_comparison_metric(dict_efficiency[level],
                               reference_competitor,
                               'energy_efficiency',
                               f"GPU Allocated Resources per watt (normalized) (set experiments: {level})",
                               'GPU Allocated Resources (in millis) per watt (normalized)')

    if False :
        plot_comparison_metric(dict_efficiency[level],
                               reference_competitor,
                               'overall_efficiency',
                               f"Overall efficiency (set experiments: {level})",
                               'Overall Efficiency')


    # Plot the difference between number of pods that failed with some competitor w.r.t. the reference competitor.
    # X-axis represents the arrived workloads in % of GPU resources available in the cluster. 
    if False :
        plot_failed_relative(dict_sched_final_res[level], reference_competitor, 
                             "failed_pods_cumsum", f"Comparison of failed pods vs {reference_competitor} (set experiments: {level})")

    # Plot the overall cluster energy consumption w.r.t. the arrived workloads in % of GPU resources available in the cluster.
    if False :
        plot_comparison_metric(dict_pwr_final_res[level],
                               "power_cluster",
                               f"Overall cluster energy consumption (set experiments: {level})",
                               "Watts")


    # Plot the cluster fragmentation ratio w.r.t. the arrived workloads in % of GPU resources available in the cluster. 
    if False :
        plot_comparison_metric(dict_frag_final_res[level], 
                               "origin_ratio",
                               f"Cluster fragmentation ratio (set experiments: {level})",
                               "Cluster fragmentation ratio")


    # Plot the GPU cluster energy consumption w.r.t. the arrived workloads in % of GPU resources available in the cluster.
    if False :
        plot_comparison_metric(dict_pwr_final_res[level],
                               "power_cluster_GPU",
                               f'GPU cluster energy consumption (set experiments: {level})',
                               "Watts")
        plot_energy_savings(dict_pwr_final_res[level], reference_competitor, 
                            "power_cluster_GPU", f"GPU power savings vs {reference_competitor} (set experiments: {level})")

    

    # Plot the CPU cluster energy consumption w.r.t. the arrived workloads in % of GPU resources available in the cluster.
    if False :
        plot_comparison_metric(dict_pwr_final_res[level],
                               "power_cluster_CPU",
                               f'CPU cluster energy consumption (set experiments: {level})',
                               "Watts")
        plot_energy_savings(dict_pwr_final_res[level], reference_competitor, 
                            "power_cluster_CPU", f"CPU cluster power savings vs {reference_competitor} (set experiments: {level})")